In [1]:
from functions.dataProcessing import *
from functions.kernels import *
from functions.models import *
from functions.acquisitionFunctions import *
from timeit import default_timer as timer

df = readData()

In [2]:
def modelFit2(par, subjD, rounds, returnPredictions=False):
    tau = np.exp(float(par[0]))
    beta = np.exp(float(par[1]))
    lam = np.exp(float(par[2]))
    parVec = [lam, lam, 1, 0.0001]
    
    Xnew = [[x, y] for y in range(8) for x in range(8)]
    nLL = [0]*len(rounds)
    utilities = None
    for r in rounds:
        roundD = np.array(subjD["round"]) == r
        chosen = np.array(subjD["chosen"])[roundD]
        chosen = chosen[1:20]
        y = np.array(subjD["z"])[roundD]
        y = y[0:20]
        x1 = np.array(subjD["x"])[roundD]
        x1 = x1[0:20]
        x2 = np.array(subjD["y"])[roundD]
        x2 = x2[0:20]
        X = np.column_stack((x1,x2))

        utilities = None
        for i in range(19):
            X1 = np.matrix(X[0:i+1,:])
            y1 = np.matrix(y[0:i+1])
            out = gpr(Xnew, parVec, X1, y1, rbf)
            utilityVec = ucb(out, [beta])
            utilityVec = utilityVec - np.max(utilityVec)
            if i==0:
                utilities = utilityVec
            else:
                utilities = np.row_stack((utilities, utilityVec))
        p = np.exp(utilities / tau)
        p = p / np.sum(p, 1)
        p[p < 0.00001] = 0.00001
        p[p > 0.99999] = 0.99999
        a = [[x, chosen[x]] for x in range(19)]
        nLL[[x for x in range(len(rounds)) if rounds[x]==r][0]] = -np.sum(np.log([p[x[0], x[1]] for x in a]))
        #clear_output(wait=True)
        #print(str(r+1) + "/" + str(len(rounds)))
    if returnPredictions:
        return {"sumLogLoss": np.sum(nLL), "predictions": p, "chosen": chosen}
    else:
        return np.sum(nLL)
    
def modelFit(par, subjD, rounds, returnPredictions=False):
    tau = float(par["tau"])
    lam = float(par["lambda"])
    beta = float(par["beta"])
    return modelFit2([tau, beta, lam], subjD, rounds, returnPredictions)

In [4]:
# Test
modelFit2([0.953290334560277,0.14184440300399,0.0706500217485385], df.iloc[0], [x for x in range(10)], False)

799.150157848398

In [3]:
import scipy.optimize as sco

def cvfunc(data, task, selector, leaveoutindex):
    d1 = [data.iloc[x] for x in range(len(data)) if data.iloc[x]["id"][0] == selector][0].copy()
    b = [d1["context"][x]==task for x in range(len(d1["context"]))]
    for i in range(len(d1)):
        d1[i] = [d1[i][x] for x in range(len(d1[i])) if b[x]]
        
    rounds = [x for x in range(9)]
    trainingSet = [rounds[x] for x in range(len(rounds)) if x != leaveoutindex]
    testSet = rounds[leaveoutindex]
    bounds = [(-5,5),(-5,5),(-5,5)]
    fit = sco.differential_evolution(modelFit2, bounds, (d1, trainingSet, False), disp=True, maxiter=20)
    paramEstimates = fit.x
    print(fit)
    
    
    predict = modelFit2(paramEstimates, d1, [testSet], True)
    cvresults = {"loo": leaveoutindex, "nLL": predict["sumLogLoss"], "tau": paramEstimates[0], "lambda": paramEstimates[1], "beta": paramEstimates[2]}
    return [cvresults, predict["predictions"], predict["chosen"]]

In [ ]:
# Cross validation
start = timer()

crossvalidation = [0]*9
modelPrediction = [np.zeros((19,64)) for x in range(9)]
chosenMatrix = np.zeros((9,19))

print("0/9")
for loo in range(9):
    cv = cvfunc(data=df, task="Conceptual", selector = 1, leaveoutindex=loo)
    crossvalidation[loo] = cv[0]
    modelPrediction[loo] = cv[1]
    chosenMatrix[loo,:] = cv[2]
    clear_output(wait=True)
    print(str(loo+1) + "/9")
end = timer()
print(end - start)